## Unsupervised Metadata Assessment

In this notebook you will find the functions and methodology used for classifying key class variables.  
The dataset used is from Gene Expression Omnibus [GEO](https://www.ncbi.nlm.nih.gov/geo/)  
        The pipeline is divided into several phases:    
     **1. In view of the need for datasets with binary classes, we proceed to separate the classes into subsets**  
     **2. The idea is to create features from a topic modeling perspective for each class**  
     **3. A matrix is formed with all cases and the value corresponding to the topic found**  
     **4. Given that each subset is unbalanced, you do not want to incur overfitting then you have a balanced**  
     **5. A logistic classifier is carried out since the variables are a matrix of weights**  
     **6. A training set evaluation is performed.**

In [1]:
import turicreate as tc

### Geo Dataset

In [2]:
sf_keys = tc.SFrame('../datasets/geo_set.csv')

Finished parsing file /home/jovyan/datasets/geo_set.csv

Parsing completed. Parsed 100 lines in 0.027627 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/jovyan/datasets/geo_set.csv

Parsing completed. Parsed 1645 lines in 0.022777 secs.

In [3]:
sf_keys

key,key_class
# of tissue = 36 tissue,tissue
4 months tissue,tissue
prostate tissue,tissue
# of tissue = 1 tissue,tissue
# of tissue = 40 tissue,tissue
# of tissue = 42 tissue,tissue
fetal vs adult tissue,tissue
# of tissue = 38 tissue,tissue
a.thaliana tissue,tissue
age of ffpe tissue,tissue


In [4]:
#import pandas as pd
#df_values = pd.read_csv('../datasets/geo_values.csv')
#df_values[df_values['values'].notna()]
#sf_values = tc.SFrame('../datasets/geo_values.csv')

In [5]:
key_classes = [i for i in sf_keys['key_class'].unique()]

In [6]:
print('Number of key value pairs: {}\nDifferent key classes: {}'.format(len(sf_keys), key_classes))

Number of key value pairs: 1645
Different key classes: ['gender', 'cell line', 'genotype', 'sex', 'treatment', 'age', 'cell type', 'strain', 'time', 'disease', 'tissue']


###  Key class subsets

It is needed to create different subsets, one per each key class, a filtering is performed

In [7]:
def create_subsets(df, column_category):
    tuples = []
    for category in df[column_category].unique():
        yes_category = df[df[column_category] == category]
        no_category = df[df[column_category] != category]
        no_category[column_category] = 'no '+category
        table = yes_category.append(no_category)
        tuples.append((category, table))
        
    tables = {key: value for (key, value) in tuples}
    print(tables.keys())
    return tables

In [8]:
geo_tables = create_subsets(sf_keys, 'key_class')

dict_keys(['gender', 'cell line', 'genotype', 'sex', 'treatment', 'age', 'cell type', 'strain', 'time', 'disease', 'tissue'])


In [9]:
for k in geo_tables.keys():
    print(str(k), round((sum(list(geo_tables[str(k)]['key_class'] == str(k)))/len(sf_keys)),2))

gender 0.03
cell line 0.05
genotype 0.07
sex 0.02
treatment 0.15
age 0.18
cell type 0.02
strain 0.11
time 0.24
disease 0.05
tissue 0.09


Here it is an example of the first 40 rows of a subset looks like

In [10]:
geo_tables['cell type'].print_rows(40,2)

+-------------------------------+--------------+
|              key              |  key_class   |
+-------------------------------+--------------+
|      pancreatic cell type     |  cell type   |
|        cancer cell type       |  cell type   |
|          cell type(s)         |  cell type   |
|       targeted cell type      |  cell type   |
|         cell type/line        |  cell type   |
|  cell type (immunophenotype)  |  cell type   |
|    cell type subpopulation    |  cell type   |
|        sorted cell type       |  cell type   |
|        cell type subset       |  cell type   |
|           cell types          |  cell type   |
|         cell type [c]         |  cell type   |
|     cell type derived from    |  cell type   |
|        cell type origin       |  cell type   |
|      background cell type     |  cell type   |
|       cell type (sorted)      |  cell type   |
|        tumor cell type        |  cell type   |
|         host cell type        |  cell type   |
|        source cell

### Pipeline method for one class

In [11]:
key_class = 'cell line'
class_column_name = 'key_class'

### Extracting features from topic modeling

In [12]:
def create_features(category_df, n_features):
    # Remove stopwords and convert to bag of words
    doc = tc.text_analytics.count_words(category_df['key'])
    doc = doc.dict_trim_by_keys(tc.text_analytics.stop_words(), True)
    
    # Learn topic model
    model = tc.topic_model.create(doc, verbose=False)
    # Agreaggate the unique words
    sf_topics = model.get_topics()
    sf_words = sf_topics.groupby(key_column_names='word', operations={'sum_scores': tc.aggregate.SUM('score')})
    
    # Sort the features scores and filter out all those which are key
    sf_words = sf_words.sort('sum_scores', ascending= False).filter_by(key_classes, 'word', exclude=True)
    
    # Take a look of the features related with this key class
    features = [i for i in sf_words['word']][0:n_features] #changable
    return sf_words, features

In [13]:
sf_words, features = create_features(geo_tables[key_class], 10)

Learning a topic model

Number of documents      1645

Vocabulary size       962

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 21.593ms      | 3.54141e+06    | 0               |

+-----------+---------------+----------------+-----------------+

### Creating a feature matrix

In [14]:
def get_input_matrix(features, category_df, sf_words):
    tuples = []
    for word in features:
        feature_vector = [1 if (word in i) else 0 for i in category_df['key']]
        tuples.append((word, feature_vector))
        
    sf_features = tc.SFrame({key: value for (key, value) in tuples})
    #concatenating the features with the category matrix
    category_df = category_df.add_row_number()
    sf_features = sf_features.add_row_number()
    final_table = category_df.join(sf_features, on='id', how='left')
    for f in features:
        score = sf_words[sf_words['word'] == str(f)]['sum_scores'].astype(float)[0]
        final_table[str(f)] = [(1.0+score) * i for i in final_table[str(f)]]
        
    return final_table

In [15]:
input_matrix = get_input_matrix(features, geo_tables[key_class], sf_words)

### Class balancing

In [16]:
def binary_balancing(sf, key_class, class_column_name):
    class_set = sf[sf[class_column_name]== key_class ]
    inverse_set = sf[sf[class_column_name]!= key_class]
    return class_set.append(inverse_set.sample(float(len(class_set))/float(len(input_matrix))))

In [17]:
balanced_input_matrix = binary_balancing(input_matrix, key_class, class_column_name)

Here it is an example of the first 10 rows of the input matrix looks like

In [18]:
balanced_input_matrix

id,key,key_class,cell,days,infection,line,months,point
0,human cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
1,dendritic cell lineages,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
2,or cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
3,huh7 cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
4,hybrid cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
5,tumor cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
6,atcc cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
7,host cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
8,donor cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0
9,fibrosarcoma cell line,cell line,1.238190701358203,0.0,0.0,1.160765616429564,0.0,0.0


### Logistic Classifier

In [19]:
train_data, test_data = balanced_input_matrix.random_split(0.65)

In [20]:
model = tc.logistic_classifier.create(train_data, target = 'key_class', features = features, max_iterations = 10)

Logistic regression:

--------------------------------------------------------

Number of examples          : 89

Number of classes           : 2

Number of feature columns   : 10

...

| 4         | 5        | 0.005987     | 1.000000          |

| 5         | 6        | 0.007757     | 1.000000          |

| 7         | 8        | 0.009217     | 1.000000          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

In [21]:
model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 11
Number of examples             : 89
Number of classes              : 2
Number of feature columns      : 10
Number of unpacked features    : 10

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 7
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0094

Settings
--------
Log-likelihood                 : 0.1948

Highest Positive Coefficients
-----------------------------
(intercept)                    : 6.7454
months                         : 1.0295
infection                      : 1.0292
point                          : 1.0274
years                          : 1.0033

Lowest Negative Coefficients
----------------------------
line                           : -7.722
cell       

In [22]:
predictions = model.classify(test_data)
evaluation_test_set = model.evaluate(test_data)

In [23]:
evaluation_test_set

{'accuracy': 1.0, 'auc': 1.0, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 2
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  cell line   |    cell line    |   31  |
 | no cell line |   no cell line  |   32  |
 +--------------+-----------------+-------+
 [2 rows x 3 columns], 'f1_score': 1.0, 'log_loss': 0.004439872350865624, 'precision': 1.0, 'recall': 1.0, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+----+----+
 | threshold | fpr | tpr | p  | n  |
 +-----------+-----+-----+----+----+
 |    0.0    | 1.0 | 1.0 | 32 | 31 |
 |   1e-05   | 1.0 | 1.0 | 32 | 31 |
 |   2e-05   | 1.0 | 1.0 | 32 | 31 |
 |   3e-05   | 1.0 | 1.0 | 32 | 31 |
 |   4e-05   | 1.0 | 1.0 | 32 | 31 |
 |   5e-05   | 1.0 | 1.0 | 32 | 31 |
 |   6e-05   | 1.0 | 1.0 | 32 | 31 |
 |   7e-05   | 1.0 | 1

---
### Pipeline methodology

In [26]:
def method_pipeline(sf, class_column_name):
    output = []
    key_classes = [i for i in sf[class_column_name].unique()]
    for key_class in key_classes:
        sf_words, features = create_features(geo_tables[key_class], 10)
        input_matrix = get_input_matrix(features, geo_tables[key_class], sf_words)
        balanced_input_matrix = binary_balancing(input_matrix, key_class, class_column_name)
        train_data, test_data = balanced_input_matrix.random_split(0.7)
        model = tc.logistic_classifier.create(train_data, target = class_column_name, features = features, max_iterations = 10, verbose=False)
        evaluation_test_set = model.evaluate(test_data)
        output.append((key_class, evaluation_test_set))
    return {key: value for (key, value) in output}    

In [27]:
results = method_pipeline(sf_keys, 'key_class')

Learning a topic model

Number of documents      1645

Vocabulary size       962

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 31.924ms      | 2.87677e+06    | 0               |

+-----------+---------------+----------------+-----------------+

In [28]:
results.keys()

dict_keys(['gender', 'cell line', 'genotype', 'sex', 'treatment', 'age', 'cell type', 'strain', 'time', 'disease', 'tissue'])

In [33]:
results['cell type']

{'accuracy': 0.9545454545454546, 'auc': 1.0, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 3
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  cell type   |    cell type    |   10  |
 | no cell type |   no cell type  |   11  |
 |  cell type   |   no cell type  |   1   |
 +--------------+-----------------+-------+
 [3 rows x 3 columns], 'f1_score': 0.9565217391304348, 'log_loss': 0.09449158474618659, 'precision': 0.9166666666666666, 'recall': 1.0, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+----+----+
 | threshold | fpr | tpr | p  | n  |
 +-----------+-----+-----+----+----+
 |    0.0    | 1.0 | 1.0 | 11 | 11 |
 |   1e-05   | 1.0 | 1.0 | 11 | 11 |
 |   2e-05   | 1.0 | 1.0 | 11 | 11 |
 |   3e-05   | 1.0 | 1.0 | 11 | 11 |
 |   4e-05   | 1.0 | 1.0 | 11 | 11 |
 |   5e-

#### Roc curve

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
roc_curve_table = results['treatment']['roc_curve']
plt.scatter(roc_curve_table['fpr'], roc_curve_table['tpr'])

In [ ]:
#test_data.export_csv('../datasets/cell line/test_data_cell_line.csv')
#predictions.export_csv('../datasets/cell line/prediction_cell_line.csv')